In [ ]:
%cd ..

/home/den/dev/git/ozon-e-cup-2025


In [ ]:
import json

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

import src.preprocessing as preproc

In [ ]:
train_ids = np.load("../data/train_ids.npy")
val_ids = np.load("../data/val_ids.npy")

# Preprocess meta features

In [ ]:
meta_train = pd.read_csv("data/1__train_with_flags.csv")
meta_test = pd.read_csv("data/1__test_with_flags.csv")

In [ ]:
meta_train["SellerID"] = meta_train["SellerID"].astype("string")
meta_test["SellerID"] = meta_test["SellerID"].astype("string")

In [ ]:
preproc_pipe = Pipeline(
    steps=[
        ("num_preproc", preproc.NUMERIC_PREP_PIPELINE),
        ("cat_preproc", preproc.CATEGORIAL_PREP_PIPELINE),
    ]
)

In [ ]:
preproc_pipe = preproc_pipe.fit(meta_train.iloc[train_ids])

In [ ]:
meta_train_preproc = preproc_pipe.transform(meta_train.iloc[train_ids], is_train=True)
meta_val_preproc = preproc_pipe.transform(meta_train.iloc[val_ids], is_train=False)
meta_test_preproc = preproc_pipe.transform(meta_test, is_train=False)

In [ ]:
meta_train = pd.concat([meta_train_preproc, meta_val_preproc], axis=0)
meta_test = meta_test_preproc

In [ ]:
meta_train.drop(columns=["description", "name_rus", "ItemID"], inplace=True)
meta_test.drop(columns=["description", "name_rus", "ItemID"], inplace=True)

In [ ]:
meta_feats_cols = meta_train.columns.tolist()
meta_feats_cols.remove("id")

# Merge with text basic features

In [ ]:
text_feats_train = pd.read_csv("data/X_train_text_features_basic.csv")
text_feats_test = pd.read_csv("data/X_test_text_features_basic.csv")

In [ ]:
text_feats_cols = text_feats_train.columns.to_list()
text_feats_cols.remove("id")

In [ ]:
set(meta_train.columns) & set(text_feats_train.columns), set(meta_test.columns) & set(text_feats_test.columns)

({'id'}, {'id'})

In [ ]:
train = pd.merge(meta_train, text_feats_train, on="id", how="left")
test = pd.merge(meta_test, text_feats_test, on="id", how="left")

In [ ]:
assert train.shape[1] - 1 == len(meta_feats_cols) + len(text_feats_cols)

In [ ]:
train.to_csv("data/train_meta_text_basic_feats.csv", index=False)
test.to_csv("data/test_meta_text_basic_feats.csv", index=False)

In [45]:
json.dump(
    {"meta_feats": meta_feats_cols, "text_feats": text_feats_cols},
    open("data/cols.json", "w"),
)